#### Import necessary libraries

In [1]:
import numpy as np 
import pandas as pd 
import requests
from bs4 import BeautifulSoup

#### Scrape Data from Wikipedia

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html5lib')

postal_codes_dict = {} # initialize an empty dictionary to save the data in
for table_cell in soup.find_all('td'):
    try:
        postal_code = table_cell.p.b.text # get the postal code
        postal_code_investigate = table_cell.span.text
        neighborhoods_data = table_cell.span.text # get the rest of the data in the cell
        borough = neighborhoods_data.split('(')[0] # get the borough in the cell
        
        # if the cell is not assigned then ignore it
        if neighborhoods_data == 'Not assigned':
            neighborhoods = []
        # else process the data and add it to the dictionary
        else:
            postal_codes_dict[postal_code] = {}
            
            try:
                neighborhoods = neighborhoods_data.split('(')[1]
            
                # remove parantheses from neighborhoods string
                neighborhoods = neighborhoods.replace('(', ' ')
                neighborhoods = neighborhoods.replace(')', ' ')

                neighborhoods_names = neighborhoods.split('/')
                neighborhoods_clean = ', '.join([name.strip() for name in neighborhoods_names])
            except:
                borough = borough.strip('\n')
                neighborhoods_clean = borough
 
            # add borough and neighborhood to dictionary
            postal_codes_dict[postal_code]['borough'] = borough
            postal_codes_dict[postal_code]['neighborhoods'] = neighborhoods_clean
    except:
        pass
    
# create an empty dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

# populate dataframe with data from dictionary
for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)


##### Here, we had to index the first column called Postal Code

In [15]:
df= pd.read_csv(body).set_index("Postal Code")
df.head()


,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park/Harbourfront
M6A,North York,Lawrence Manor/Lawrence Heights
M7A,Downtown Toronto,Queen's Park/Ontario Provincial Government


#### Retrieve Postal Coordinates
##### Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.  Here, I aquired the latitude and longitude coordinates and appended them to a new data frame.

In [16]:
toronto_geo='https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geo
geocsv_data = pd.read_csv(toronto_geo).set_index("Postal Code")
geocsv_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


#### Join the coordinate data frame and our orginial data frame using the .join()

In [18]:
toronto_neighborhoods = df.join(geocsv_data)
toronto_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,Malvern/Rouge,43.806686,-79.194353
M1C,Scarborough,Rouge Hill/Port Union/Highland Creek,43.784535,-79.160497
M1E,Scarborough,Guildwood/Morningside/West Hill,43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Use the .shape() to find the dimensions of our combined data frame

In [19]:
toronto_neighborhoods.shape

(103, 4)

In [ ]:
#Our new dataframe has 103 rows with 4 columns